In [3]:
import gym
import gym_chess

# Create the chess environment
env = gym.make('Chess-v0')

# Reset the environment (initialize the board)
state = env.reset()

# Display the initial board state
print(env.render())

# Sample a random action (move) from the list of legal moves
action = env.action_space.sample()

# Take that action in the environment and receive feedback
next_state, reward, done, info = env.step(action)

# Display the new board state after the move
print(env.render())

# Check if the game is over
if done:
    print("Game over!")
else:
    print("Game continues.")


♜ ♞ ♝ ♛ ♚ ♝ ♞ ♜
♟ ♟ ♟ ♟ ♟ ♟ ♟ ♟
⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘
⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘
⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘
⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘
♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙
♖ ♘ ♗ ♕ ♔ ♗ ♘ ♖


AttributeError: 'NoneType' object has no attribute 'sample'